### Before running, make sure that Graphviz is installed on your PC and is added to system path
### https://graphviz.gitlab.io/download/

In [1]:
# IMPORTS
import os
import ipywidgets as widgets
import graphviz
import sqlite3

from subprocess import Popen, PIPE as P

In [2]:
# Path to SQLITE db generated by noWorkFlow
NOW2_SQLITE_PATH = 'C:/Users/linha/Desktop/ws/py-scripts-examples/age-avg/.noworkflow/db.sqlite'

In [3]:
class DotDisplay:

    def __init__(self, dot):
        self.dot = dot
    
    def _ipython_display_(self):
        from IPython.display import display
        bundle = {}

        dot = self.dot
        bundle['text/vnd.graphviz'] = dot

        try:
            kwargs = {} if os.name != 'nt' else {"creationflags": 0x08000000}
            p = Popen(['dot', '-T', "svg"], stdout=P, stdin=P, stderr=P, **kwargs)
            image = p.communicate(dot.encode('utf-8'))[0]
            bundle['image/svg+xml'] = image.decode("utf-8")
        except OSError as e:
            if e.errno != os.errno.ENOENT:
                raise

        #bundle['text/plain'] = '\n'.join(map(str, self.log))
        display(bundle, raw=True)

In [4]:
from debugprov.node import Node
from debugprov.validity import Validity
from debugprov.provenance_navigation import ProvenanceNavigation
from ipywidgets import VBox, Output

class JNNavigationStrategy(ProvenanceNavigation):
    
    def __init__(self, *args, **kwargs):
        super(JNNavigationStrategy, self).__init__(*args, **kwargs)
        #self.joutput = Output()
        
    #@property
    #def view(self):
    #    return self.joutput
    
    def evaluate(self, node: Node) -> Node:
        self.exec_tree.node_under_evaluation = node
        vis = Visualization(self.exec_tree)
        vis.generate_exec_tree(str(id(node)))
        #self.joutput.clear_output()
        #with self.joutput:
        print(str(vis.graph))
        display(DotDisplay(str(vis.graph)))
        print("-------------------------")
        print("Evaluating node {}".format(node.name))
        print("Name: {}".format(node.name))
        print("Evaluation_id: {}".format(node.id))
        print("Code_component_id: {}".format(node.code_component_id))
        print("Parameters: name | value ")
        for p in node.params:
            print (" {} | {} ".format(p.name, p.value))
        print("Returns: {}".format(node.retrn))
        ans = input("Is correct? Y/N ")
        if ans == "Y" or ans == "y":
            # The YES answer prunes the subtree rooted at N
            self.recursive_validate(node)
        else:
            # The NO answer prunes all the nodes of the ET,
            # exept the subtree rooted at N
            node.validity = Validity.INVALID
            if node.parent is not None:
                for c in node.parent.childrens:
                    if c is not node:
                        self.recursive_validate(c)
            self.exec_tree.node_under_evaluation = None
        return node

ImportError: No module named 'debugprov.provenance_navigation'

In [5]:
from debugprov.execution_tree_creator import ExecTreeCreator
from debugprov.top_down import TopDown
from debugprov.heaviest_first import HeaviestFirst
from debugprov.visualization import Visualization
from debugprov.provenance_navigation import ProvenanceNavigation
from debugprov.single_stepping import SingleStepping
from debugprov.divide_and_query import DivideAndQuery

ImportError: No module named 'debugprov.provenance_navigation'

In [6]:
CURSOR = sqlite3.connect(NOW2_SQLITE_PATH).cursor()
creator = ExecTreeCreator(CURSOR)
exec_tree = creator.create_exec_tree()
nav = JNNavigationStrategy(exec_tree, CURSOR)
result_tree = nav.navigate()
vis = Visualization(result_tree)
vis.view_exec_tree_prov('exec_tree',nav.DEPENDENCIES)

NameError: name 'JNNavigationStrategy' is not defined

In [ ]:
#o = Output()
#with o:
#    print(1234)

In [ ]:
#o

In [ ]:
#o.clear_output()